In [11]:
import numpy as np 
import pandas as pd 

In [12]:
df_train_x = pd.read_csv('train.csv')
df_train_y = df_train_x[['label']]
df_train_x = df_train_x.drop("label", axis=1)
df_test_x = pd.read_csv('test.csv')
df_test_x

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
df_train_x = np.array(df_train_x)
df_test_x = np.array(df_test_x)

In [14]:
df_train_x = np.reshape(df_train_x, (df_train_x.shape[0], 28,28,1))
df_test_x = np.reshape(df_test_x, (df_test_x.shape[0], 28,28,1))

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
x_train, x_test, y_train, y_test = train_test_split(df_train_x, df_train_y, test_size = 0.25, random_state=42, stratify=df_train_y)

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

In [30]:
model = Sequential()
model.add(Conv2D(32, kernel_size = 3, activation = 'relu', input_shape = (df_train_x.shape[1],df_train_x.shape[2],df_train_x.shape[3])))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation = 'softmax'))
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
early = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [31]:
model.fit(x_train, y_train, validation_split=0.3, epochs=50, shuffle=True, callbacks=[early])


Epoch 1/50
690/690 [==============================] - 13s 18ms/step - loss: 3.6641 - accuracy: 0.6779 - val_loss: 0.1987 - val_accuracy: 0.9423
Epoch 2/50
690/690 [==============================] - 12s 18ms/step - loss: 0.1348 - accuracy: 0.9589 - val_loss: 0.1612 - val_accuracy: 0.9553
Epoch 3/50
690/690 [==============================] - 12s 18ms/step - loss: 0.0652 - accuracy: 0.9804 - val_loss: 0.1583 - val_accuracy: 0.9602
Epoch 4/50
690/690 [==============================] - 12s 18ms/step - loss: 0.0303 - accuracy: 0.9903 - val_loss: 0.1906 - val_accuracy: 0.9554
Epoch 5/50
690/690 [==============================] - 12s 18ms/step - loss: 0.0217 - accuracy: 0.9932 - val_loss: 0.2093 - val_accuracy: 0.9595
Epoch 6/50
690/690 [==============================] - 12s 18ms/step - loss: 0.0152 - accuracy: 0.9946 - val_loss: 0.1913 - val_accuracy: 0.9625
Epoch 7/50
690/690 [==============================] - 12s 18ms/step - loss: 0.0049 - accuracy: 0.9986 - val_loss: 0.2291 - val_accuracy:

In [32]:
print(model.evaluate(x_train, y_train))
print(model.evaluate(x_test, y_test))

985/985 [==============================] - 5s 5ms/step - loss: 0.0663 - accuracy: 0.9823
[0.06628671288490295, 0.9823492169380188]
329/329 [==============================] - 2s 5ms/step - loss: 0.1657 - accuracy: 0.9557
[0.1657351702451706, 0.9557142853736877]


In [34]:
prediction = model.predict(df_test_x)

In [38]:
final_predictions = np.argmax(prediction,axis=1)

In [41]:
df_test_x = pd.read_csv('test.csv')
df_test_x.shape

(28000, 784)

In [43]:
final_df = pd.DataFrame({"ImageId":np.arange(1,df_test_x.shape[0]+1), 'Label':final_predictions})
final_df.to_csv('MNIST.csv', index=None)